In [1]:
import pandas as pd
import os
import requests
import time
import numpy as np

In [2]:
path="D:\\CDS513\\Assignment 1\\Data"
    
os.chdir(path)

# Read csv data file
# Data without feature standardization
df = pd.read_csv('steam-200k.csv', header=None)

In [3]:
df

,0,1,2,3,4
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,0


In [4]:
df.columns = [
    'User-id',
    'Game-title',
    'Mode',
    'Playtime',
    'misc'
]
df.set_index('User-id', inplace=True)
df

,Game-title,Mode,Playtime,misc
User-id,,,,
151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
151603712,The Elder Scrolls V Skyrim,play,273.0,0
151603712,Fallout 4,purchase,1.0,0
151603712,Fallout 4,play,87.0,0
151603712,Spore,purchase,1.0,0
...,...,...,...,...
128470551,Titan Souls,play,1.5,0
128470551,Grand Theft Auto Vice City,purchase,1.0,0
128470551,Grand Theft Auto Vice City,play,1.5,0


In [5]:
df['Mode'].value_counts()

purchase    129511
play         70489
Name: Mode, dtype: int64

In [6]:
# Pivot the DataFrame to transform 'purchase' and 'play' modes into separate columns
pivot_df = df.pivot_table(index=['User-id', 'Game-title'], columns='Mode', values='Playtime', fill_value=0)

# Reset the index to turn multi-index into columns
pivot_df = pivot_df.reset_index()

# Rename columns for clarity
pivot_df.columns = ['User-id', 'Game-title', 'Playtime', 'Purchase']

# Display the transformed DataFrame
pivot_df.head()


,User-id,Game-title,Playtime,Purchase
0,5250,Alien Swarm,4.9,1
1,5250,Cities Skylines,144.0,1
2,5250,Counter-Strike,0.0,1
3,5250,Counter-Strike Source,0.0,1
4,5250,Day of Defeat,0.0,1


In [7]:
game_titles = pivot_df['Game-title'].unique().tolist()

In [8]:
game_titles

['Alien Swarm',
 'Cities Skylines',
 'Counter-Strike',
 'Counter-Strike Source',
 'Day of Defeat',
 'Deathmatch Classic',
 'Deus Ex Human Revolution',
 'Dota 2',
 'Half-Life',
 'Half-Life 2',
 'Half-Life 2 Deathmatch',
 'Half-Life 2 Episode One',
 'Half-Life 2 Episode Two',
 'Half-Life 2 Lost Coast',
 'Half-Life Blue Shift',
 'Half-Life Opposing Force',
 'Portal',
 'Portal 2',
 'Ricochet',
 'Team Fortress 2',
 'Team Fortress Classic',
 'Age of Empires II HD Edition',
 'Arma 2',
 'Arma 2 Operation Arrowhead',
 'Arma 2 Operation Arrowhead Beta (Obsolete)',
 'Banished',
 'Call of Duty Black Ops',
 'Call of Duty Black Ops - Multiplayer',
 'Call of Duty Modern Warfare 2',
 'Call of Duty Modern Warfare 2 - Multiplayer',
 'Call of Duty Modern Warfare 3',
 'Call of Duty Modern Warfare 3 - Multiplayer',
 'Call of Duty World at War',
 'Counter-Strike Global Offensive',
 'Rise of Nations Extended Edition',
 'The Stanley Parable',
 'Thief',
 'Thief - Ghost',
 'Thief - Opportunist',
 'Thief - Preda

In [9]:
np.count_nonzero(game_titles)

5155

# Getting Genres for each title

In [ ]:
# IGDB API endpoint for games
url = 'https://api.igdb.com/v4/games'
client_id = 'sxiuj4y04yliue4d66h7dkoxt3sjjf'
access_token = 'vbc6nonqwnvym37xdlzfa2df8sh07i'
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json'
}

In [ ]:
def search_games(titles, batch_size=600):
    # Prepare to collect genres
    genres_dict = {}

    # Process in batches to manage API request volume
    for i in range(0, len(titles), batch_size):
        batch_titles = titles[i:i+batch_size]
        # Constructing the body for multiple searches can be complex because IGDB API may not support multiple searches in one request.
        # Here we handle them one by one in a batch loop for demonstration.
        for title in batch_titles:
            body = f'search "{title}"; fields name, genres.name; limit 100;'
            response = requests.post(url, headers=headers, data=body)
            if response.status_code == 200:
                data = response.json()
                if data and 'genres' in data[0] and data[0]['genres']:
                    genres_dict[title] = data[0]['genres'][0]['name']
                else:
                    genres_dict[title] = 'Genre not found'
            else:
                print(f'API request failed with status code {response.status_code}')
            time.sleep(1)  # Sleep to respect API rate limit
            
    return genres_dict

In [ ]:
genres_info = search_games(game_titles)
print(genres_info)

In [ ]:
genres_df = pd.DataFrame.from_dict(genres_info, orient='index', columns=['Genre'])

genres_df.reset_index(inplace=True)

genres_df.rename(columns={'index': 'Game-title'}, inplace=True)


In [ ]:
genres_df

In [ ]:
# Concatenate with the existing DataFrame
combined_genres_df = pd.concat([combined_genres_df, genres_df], ignore_index=True)

In [ ]:
combined_genres_df

In [42]:
combined_genres_df.drop_duplicates()

,Game-title,Genre
0,Alien Swarm,Shooter
1,Cities Skylines,Simulator
2,Counter-Strike,Shooter
3,Counter-Strike Source,Shooter
4,Day of Defeat,Shooter
...,...,...
7150,Heroes Never Lose Professor Puzzler's Perplexi...,Puzzle
7151,Operation Z,Indie
7152,Retaliation,Adventure
7153,Abducted,Role-playing (RPG)


In [33]:
combined_genres_df.to_csv('game_genres.csv', index=False)

In [34]:
combined_genres_df = pd.read_csv('game_genres.csv')

In [35]:
pivot_df

,User-id,Game-title,Playtime,Purchase
0,5250,Alien Swarm,4.9,1
1,5250,Cities Skylines,144.0,1
2,5250,Counter-Strike,0.0,1
3,5250,Counter-Strike Source,0.0,1
4,5250,Day of Defeat,0.0,1
...,...,...,...,...
128799,309626088,Age of Empires II HD Edition,6.7,1
128800,309812026,Counter-Strike Nexon Zombies,0.0,1
128801,309812026,Robocraft,0.0,1
128802,309824202,Dota 2,0.7,1


In [38]:
merged_df = pd.merge(pivot_df, combined_genres_df, on='Game-title', how='left')
merged_df = merged_df.drop_duplicates()

In [39]:
merged_df

,User-id,Game-title,Playtime,Purchase,Genre
0,5250,Alien Swarm,4.9,1,Shooter
2,5250,Cities Skylines,144.0,1,Simulator
4,5250,Counter-Strike,0.0,1,Shooter
6,5250,Counter-Strike Source,0.0,1,Shooter
8,5250,Day of Defeat,0.0,1,Shooter
...,...,...,...,...,...
228802,309626088,Age of Empires II HD Edition,6.7,1,Real Time Strategy (RTS)
228804,309812026,Counter-Strike Nexon Zombies,0.0,1,Genre not found
228806,309812026,Robocraft,0.0,1,Shooter
228808,309824202,Dota 2,0.7,1,Strategy


In [29]:
cleaned_df = merged_df[merged_df['Genre'] != 'Genre not found'].drop(['Purchase'],axis=1)

In [30]:
cleaned_df

,User-id,Game-title,Playtime,Genre
0,5250,Alien Swarm,4.9,Shooter
2,5250,Cities Skylines,144.0,Simulator
4,5250,Counter-Strike,0.0,Shooter
6,5250,Counter-Strike Source,0.0,Shooter
8,5250,Day of Defeat,0.0,Shooter
...,...,...,...,...
228801,309554670,Mitos.is The Game,5.9,Real Time Strategy (RTS)
228802,309626088,Age of Empires II HD Edition,6.7,Real Time Strategy (RTS)
228806,309812026,Robocraft,0.0,Shooter
228808,309824202,Dota 2,0.7,Strategy


# Deriving Ratings

In [ ]:
#Ratings = (total game playtime / total game owners) * 5 to normalize
for item in combined_genres_df['Game-title']:
    

In [53]:
cleaned_df['Game-title'].value_counts()['Dota 2']


4841